In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat May 12 15:08:40 2018

@author: Allay
"""
# import libraries
import csv
import pickle
import tensorflow as tf
import numpy as np
import cv2
from keras.layers import Input, Flatten, Dense, Lambda
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import os, sys
import settings

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

# define flags:
#flags = tf.app.flags
#FLAGS = flags.FLAGS

#flags.DEFINE_string('data_path', '', 'Path to data folder')
#flags.DEFINE_string('training_file', '', 'Pickle file of training data')
#flags.DEFINE_string('validation_file', '', 'Pickle file of validation data')
#flags.DEFINE_integer('epochs', 1, 'Number of epochs')


# Load file paths
def LoadRawData(path):
    with open(path+'driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        lines = [line for line in reader]
    # load data: images and measurements
    images = np.array([cv2.imread(line[0]) for line in lines])
    measurements = np.array([float(line[3]) for line in lines])
    return images, measurements

def PrintStats():
    n_train = X_train.shape[0]
    n_test = X_test.shape[0]
    #n_test = X_test.shape[0]
    image_shape = X_train.shape[1:]
    n_classes, counts = np.unique(y_train, return_counts=True)

    print("Number of training examples =", n_train)
    #print("Number of validation examples =", n_val)
    print("Number of testing examples =", n_test)
    print("Image data shape =", image_shape)
    print("Labels data shape =", y_train.shape)
    
def LoadFromPickleFile():
    print('Loading data from pickle file...')
    for filename in os.listdir(settings.DATA_PATH):
        if filename == settings.DATA_PICKLE_FILENAME:
            try:
                with open(settings.DATA_PATH + filename, 'rb') as f:
                    data = pickle.load(f)
                    return data['train_dataset'], data['train_labels'], data['test_dataset'], data['test_labels']
            except:
                print('Unable to load training data from pickle file')
                raise
            
    
def SaveDataToPickleFile():
    print('Saving data to pickle file...')
    try:
        with open(settings.DATA_PATH + settings.DATA_PICKLE_FILENAME, 'wb') as pfile:
            pickle.dump(
                {
                    'train_dataset': X_train,
                    'train_labels': y_train,
                    'test_dataset': X_test,
                    'test_labels': y_test,
                },
                pfile, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save data to', settings.DATA_PICKLE_FILENAME, ':', e)
        raise
    


C:\Users\Allay\Anaconda3\envs\gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#def main(_):
# load data
#check if pickle file exists
if os.path.isfile(settings.DATA_PATH + settings.DATA_PICKLE_FILENAME):
    X_train, y_train, X_test, y_test = LoadFromPickleFile()
else:
    X_train, y_train = LoadRawData(settings.DATA_PATH)#FLAGS.data_path)
    # Split data 
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.15, random_state=0)
    # Create pickle file for future use
    SaveDataToPickleFile()
    print('Data cached in pickle file.')

PrintStats()

# pre-process data

# define model
model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))
model.add(Flatten())
model.add(Dense(1)) 



Loading data from pickle file...
Number of training examples = 5354
Number of testing examples = 945
Image data shape = (160, 320, 3)
Labels data shape = (5354,)


In [3]:
model.compile(loss='mse', optimizer='adam',  metrics=['accuracy'])
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=2)#settings.EPOCHS)

print('Saving model...')
model.save(settings.MODEL_FILENAME)


    
# parses flags and calls the `main` function above
#if __name__ == '__main__':
#    tf.app.run()

Train on 4283 samples, validate on 1071 samples
Epoch 1/2
4283/4283 [==============================] - 3s 779us/step - loss: 6.0444 - acc: 0.2071 - val_loss: 0.4242 - val_acc: 0.4127
Epoch 2/2
4283/4283 [==============================] - 3s 757us/step - loss: 1.2732 - acc: 0.2848 - val_loss: 0.6986 - val_acc: 0.3688
Saving model...


In [4]:
model.metrics_names

['loss', 'acc']

In [5]:
print("Testing...")
metrics = model.evaluate(X_test, y_test)

for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))

Testing...
945/945 [==============================] - 1s 562us/step
loss: 0.6688937895827823
acc: 0.38941798963874735


In [6]:
from keras.models import load_model